In [1]:
%load_ext autoreload
%autoreload 2

### imports and utilities

In [2]:
!pip install nltk scikit-learn matplotlib scipy -q

In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11001] getaddrinfo failed>


False

In [65]:
from collections import Counter
from utils import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from scipy.special import softmax
from scipy.stats import norm
from scipy.stats import entropy as calculate_entropy

from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### load dataset

In [6]:
dataset = "newsgroup"

# total number of samples needed
randomize = False

# retrieve dataset
categories = ['rec.autos', 'talk.politics.mideast', 'alt.atheism', 'sci.space']

all_docs = fetch_20newsgroups(subset='train', shuffle=randomize, remove=('headers', 'footers', 'quotes'), categories=categories)
all_docs, old_labels, categories = all_docs.data, all_docs.target, all_docs.target_names

In [7]:
# dataset = "bbc"

# data = pd.read_csv('bbcsport.csv')

# all_docs = data["text"].to_list()
# old_labels = data["topic"].to_list()
# categories = classes = np.unique(data["topic"]).tolist()

### clean dataset

In [8]:
datasize = 40
min_document_length = 160
max_document_length = 256


index = -1
docs, labels, label_indices = [], [], []

sizes = [0]*len(categories)

with tqdm(total=len(categories)*datasize) as pbar:
    while sum(sizes) < len(categories)*datasize:
        index += 1
        label_index = old_labels[index]
            
        if sizes[label_index] == datasize:
            continue
        
        doc = all_docs[index]
        status, doc, word_count = clean_doc(doc, True)
        
        if not status:
            continue
            
        if min_document_length is not None and len(doc) < min_document_length:
            continue
            
        if max_document_length is not None and len(doc) > max_document_length:
            continue
        
        label_indices.append(label_index)
        labels.append(categories[label_index])
        
        docs.append(doc)
        sizes[label_index] += 1
        pbar.update(1)

labels = np.array(labels)
label_indices = np.array(label_indices)

In [9]:
doc_index = 3
print(f"Topic: {labels[doc_index]}\n{'='*50}\n{docs[doc_index][:512]}")

Topic: rec.autos
not to mention my friend s 54 citroen traction avant with the light switch and dimmer integrate in a single stalk off the steer column those dumb french be apparently copying the japanese before the german


In [10]:
print(sizes)
assert min(sizes) == max(sizes) == datasize

[40, 40, 40, 40]


### Split data

In [11]:
x_train, x_test, y_train, y_test = train_test_split(docs, labels, test_size =.3)

In [12]:
print(f"there are {len(docs)} total docs, {len(y_train)} train and {len(y_test)} test")

there are 160 total docs, 112 train and 48 test


### Initialize Vectorizer

In [13]:
# initialize the count vectorizer
vectorizer = CountVectorizer()

# fit it to dataset
vectorizer.fit(x_train)

vocabulary = np.array(vectorizer.get_feature_names())
print("word_count is", len(vocabulary))

word_count is 1467


### Prepare Datatset

In [256]:
# create doc count vectors
train_doc_vectors = vectorizer.transform(x_train).toarray()
test_doc_vectors = vectorizer.transform(x_test).toarray()

wdf_train = pd.DataFrame(train_doc_vectors, columns=vocabulary)
wdf_test = pd.DataFrame(test_doc_vectors, columns=vocabulary)

### some new method

In [257]:
wdf_train.head()

,000,031349,10,1000,11,14,16th,1900,1940,1982,...,yahud,yeah,year,yes,yo,yorker,you,your,yourself,zeuge
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [258]:
wdt_train = wdf_train.copy()
wdt_test = wdf_test.copy()

wdt_test["__labels__"] = y_test
wdt_train["__labels__"] = y_train

word_doc_count = wdf_train.sum(0)
word_word_pr_distr = pd.DataFrame(data=0.0, columns=vocabulary, index=vocabulary)

for word in tqdm(vocabulary):
    pxy = wdf_train[wdf_train[word] > 0].sum(0) / word_doc_count[word]
#     pxy = (wdf_train.T * wdf_train[word]).sum(1)/ word_doc_count[word]
    word_word_pr_distr[word] = pxy * (word_doc_count[word] / word_doc_count)

print(f"word_word_pr_distr shape = {word_word_pr_distr.shape}")


word_word_pr_distr shape = (1467, 1467)


In [259]:
word_word_pr_distr.head()

,000,031349,10,1000,11,14,16th,1900,1940,1982,...,yahud,yeah,year,yes,yo,yorker,you,your,yourself,zeuge
000,1.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0
031349,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
10,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [265]:
entropy_values = calculate_entropy(word_word_pr_distr, axis=0) * calculate_entropy(word_word_pr_distr, axis=1)
entropy_values_norm = (entropy_values.max() - entropy_values) / entropy_values.max()
entropy_values_norm = pd.DataFrame(data=entropy_values_norm, index=word_word_pr_distr.columns)[0]
entropy_values_norm.head()

000       0.658496
031349    0.762590
10        0.773685
1000      0.739970
11        0.688549
Name: 0, dtype: float64

In [266]:
y_train[:10]

array(['rec.autos', 'talk.politics.mideast', 'talk.politics.mideast',
       'rec.autos', 'rec.autos', 'sci.space', 'alt.atheism', 'rec.autos',
       'alt.atheism', 'sci.space'], dtype='<U21')

In [282]:
doc_indices = [0, 3, 1, 2, 5, 9, 6, 8]

for doc_index in doc_indices:
    print(f"Topic: {y_train[doc_index]}\n{'='*50}\n{x_train[doc_index][:512]}")
    print()

Topic: rec.autos
not to mention my friend s 54 citroen traction avant with the light switch and dimmer integrate in a single stalk off the steer column those dumb french be apparently copying the japanese before the german

Topic: rec.autos
right in the thirty both buick and packard have two spare mount in well in the front fender of course that be back when the front fender be long enough to provide room there be a couple of other marque that do this a well but memory fade

Topic: talk.politics.mideast
to my fellow columbian i must ask why do you say that i engage in fantasy arafat be a terrorist who happens to have a lot of pull among palestinian can we ignore the two fact i doubt it

Topic: talk.politics.mideast
i have read that there will be some concrete proposal concern creation of a palestinian police force during the talk s next stage do anyone know of the detail of this idea how do it fit with the differ conception list above

Topic: sci.space
actually the ether stuff sound a 

In [283]:
doc_words_li = []

for doc_index in range(len(y_train)):
    doc_words_li.append(wdf_train.iloc[doc_index][wdf_train.iloc[doc_index] > 0].index.tolist())
    
    if doc_index in doc_indices:
        print(doc_words_li[-1])

['54', 'and', 'apparently', 'avant', 'be', 'before', 'citroen', 'column', 'copying', 'dimmer', 'dumb', 'french', 'friend', 'german', 'in', 'integrate', 'japanese', 'light', 'mention', 'my', 'not', 'off', 'single', 'stalk', 'steer', 'switch', 'the', 'those', 'to', 'traction', 'with']
['among', 'arafat', 'ask', 'be', 'can', 'columbian', 'do', 'doubt', 'engage', 'fact', 'fantasy', 'fellow', 'happens', 'have', 'ignore', 'in', 'it', 'lot', 'must', 'my', 'of', 'palestinian', 'pull', 'say', 'terrorist', 'that', 'the', 'to', 'two', 'we', 'who', 'why', 'you']
['above', 'anyone', 'be', 'conception', 'concern', 'concrete', 'creation', 'detail', 'differ', 'do', 'during', 'fit', 'force', 'have', 'how', 'idea', 'it', 'know', 'list', 'next', 'of', 'palestinian', 'police', 'proposal', 'read', 'some', 'stage', 'talk', 'that', 'the', 'there', 'this', 'will', 'with']
['and', 'back', 'be', 'both', 'buick', 'but', 'couple', 'course', 'do', 'enough', 'fade', 'fender', 'front', 'have', 'in', 'long', 'marque'

In [286]:
dfx_li = []

for doc_index1 in tqdm(range(len(y_train))):
    dfx_li.append([])
    
    if doc_index1 not in doc_indices:
        continue
        
    for doc_index2 in range(len(y_train)):
        dfx = pd.DataFrame(data=0.0, index=doc_words_li[doc_index1], columns=doc_words_li[doc_index2])

        for word in doc_words_li[doc_index1]:
            for other_word in doc_words_li[doc_index2]:
                v1 = word_word_pr_distr[word][other_word] * entropy_values_norm[word]
                v2 = word_word_pr_distr[other_word][word] * entropy_values_norm[other_word]
                
                dfx[other_word][word] = min(v1, v2)
#                 dfx[other_word][word] =  v1 * v2
        
        dfx_li[-1].append(dfx)
dfx.head()

,and,bastard,be,definitely,for,ghod,have,just,nls,of,...,sub,suitable,thank,that,the,thought,use,version,will,would
be,0.003223,0.004244,0.004244,0.004244,0.003559,0.004244,0.003334,0.003591,0.004244,0.003506,...,0.004244,0.004244,0.004244,0.003380,0.0,0.004244,0.004244,0.004244,0.003994,0.003970
engineering,0.000000,0.000000,0.004244,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015090,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.022392
get,0.028831,0.000000,0.003448,0.000000,0.073965,0.000000,0.014016,0.043440,0.000000,0.027704,...,0.000000,0.000000,0.000000,0.054875,0.0,0.000000,0.053689,0.000000,0.069254,0.028483
glad,0.000000,0.000000,0.004244,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017344,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
griffin,0.000000,0.000000,0.004244,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017344,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [287]:
y_train[doc_indices]

array(['rec.autos', 'rec.autos', 'talk.politics.mideast',
       'talk.politics.mideast', 'sci.space', 'sci.space', 'alt.atheism',
       'alt.atheism'], dtype='<U21')

In [288]:
dfx_li[0][0].head()

,54,and,apparently,avant,be,before,citroen,column,copying,dimmer,...,off,single,stalk,steer,switch,the,those,to,traction,with
54,0.760858,0.009631,0.380429,0.760858,0.004113,0.380429,0.760858,0.760858,0.760858,0.760858,...,0.152172,0.380429,0.760858,0.190214,0.760858,0.0,0.253619,0.008270,0.760858,0.030434
and,0.009631,0.076883,0.008460,0.009631,0.003223,0.024921,0.009631,0.009631,0.009631,0.009631,...,0.013076,0.016616,0.009631,0.014305,0.009631,0.0,0.015848,0.031780,0.009631,0.036904
apparently,0.380429,0.008460,0.668354,0.380429,0.002122,0.328131,0.380429,0.380429,0.380429,0.380429,...,0.133671,0.328166,0.380429,0.167089,0.380429,0.0,0.222785,0.021794,0.380429,0.026734
avant,0.760858,0.009631,0.380429,0.760858,0.004113,0.380429,0.760858,0.760858,0.760858,0.760858,...,0.152172,0.380429,0.760858,0.190214,0.760858,0.0,0.253619,0.008270,0.760858,0.030434
be,0.004113,0.003223,0.002122,0.004113,0.004244,0.004244,0.004113,0.004113,0.004113,0.004113,...,0.004244,0.004244,0.004113,0.004244,0.004113,0.0,0.004244,0.003552,0.004113,0.003225


In [295]:
for doc_index1 in range(len(y_train)):
    if doc_index1 not in doc_indices:
        continue
        
    print(y_train[doc_index1])
    
    values = []
    for doc_index2 in range(len(y_train)):
        v1 = dfx_li[doc_index1][doc_index2].max(0).mean()
        v2 = dfx_li[doc_index1][doc_index2].max(1).mean()
        vx = (v1+v2)/2
        
#         print(f" {y_train[doc_index2][:15]:16s} {v1:.4f} {v2:.4f} : {vx:.4f}")
        values.append([y_train[doc_index2], v1, v2, vx])
    print(pd.DataFrame(values, columns=["topic", "v1", "v2", "vx"]).sort_values(ascending=False, by="vx").head(10))
    print()

rec.autos
                    topic        v1        v2        vx
0               rec.autos  0.561979  0.561979  0.561979
94              rec.autos  0.267870  0.290988  0.279429
29              sci.space  0.247602  0.276597  0.262100
27              rec.autos  0.225102  0.296645  0.260874
65            alt.atheism  0.193846  0.279924  0.236885
98            alt.atheism  0.191573  0.281307  0.236440
44  talk.politics.mideast  0.193085  0.279272  0.236179
35              rec.autos  0.196764  0.207807  0.202285
52  talk.politics.mideast  0.184085  0.200403  0.192244
22              rec.autos  0.167751  0.172182  0.169967

talk.politics.mideast
                     topic        v1        v2        vx
1    talk.politics.mideast  0.453370  0.453370  0.453370
106  talk.politics.mideast  0.241387  0.243954  0.242670
102  talk.politics.mideast  0.227004  0.234519  0.230762
105            alt.atheism  0.230515  0.224591  0.227553
11               rec.autos  0.223538  0.221662  0.222600
2    talk

,0
000,0.658496
031349,0.762590
10,0.773685
1000,0.739970
11,0.688549


In [253]:
entropy_values_norm.loc["be"]

0    0.004244
Name: be, dtype: float64

In [243]:
(dfx_li[0][1].max(1) * entropy_values_norm).min()#.sort_values(ascending=False)

0.0

In [210]:
wdf_train.iloc[idx2].sort_values(ascending=True)

000         0
pity        0
pimentel    0
pilot       0
piece       0
           ..
to          2
fuel        2
how         2
the         2
be          3
Name: 7, Length: 1467, dtype: int64

In [186]:
dfx.max(0).sort_values(ascending=True).head(15)

memory     0.050847
thirty     0.050847
buick      0.050847
room       0.050847
packard    0.050847
fade       0.050847
fender     0.050847
marque     0.050847
spare      0.067797
course     0.067797
mount      0.067797
front      0.067797
two        0.080000
both       0.107317
when       0.120000
dtype: float64

In [183]:
dfx.sum(0).sort_values(ascending=False).head(15)

the       4.303757
be        3.994595
to        3.977974
and       3.803540
in        3.734835
of        3.445151
that      2.753275
have      2.407538
do        2.102149
this      2.010799
but       1.688663
there     1.236046
well      0.891024
couple    0.836783
long      0.835313
dtype: float64

In [129]:
result = []
doc_index = 3
print(f"Topic: {labels[doc_index]}\n{'='*50}\n{docs[doc_index][:512]}")

width = 0
for i, word in enumerate(wdf_train.columns):
    result.append(gnbs[word].predict_log_proba(wdf_test.loc[[doc_index]])[0])
    if len(result) > width:
        width = len(result)

res = pd.DataFrame(data=result, index=wdf_train.columns).fillna(0)

Topic: rec.autos
not to mention my friend s 54 citroen traction avant with the light switch and dimmer integrate in a single stalk off the steer column those dumb french be apparently copying the japanese before the german


In [134]:
res.sum(1).sort_values()

and     -5.121664e+10
the     -5.065995e+10
mount   -3.989702e+10
spare   -3.989702e+10
issue   -3.711350e+10
             ...     
make    -1.855667e+09
we      -1.855666e+09
at      -1.670100e+09
me      -1.670099e+09
get     -1.113398e+09
Length: 1467, dtype: float64

In [113]:
res[1].sort_values(ascending=True).head(20)

be          -4.082481e+09
the         -7.422644e+08
that        -5.567038e+08
you         -3.711371e+08
of          -1.855654e+08
not         -4.879017e+03
play         0.000000e+00
planetary    0.000000e+00
plane        0.000000e+00
placement    0.000000e+00
place        0.000000e+00
pity         0.000000e+00
pimentel     0.000000e+00
pilot        0.000000e+00
piece        0.000000e+00
pickup       0.000000e+00
phone        0.000000e+00
phobos       0.000000e+00
peter        0.000000e+00
pertains     0.000000e+00
Name: 0, dtype: float64

In [99]:
np.log(0)

C:\Program Files\Python37\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


-inf

In [16]:
n_clusters = 4

wdf_train_prime = wdf_train.copy()
wdf_test_prime = wdf_test.copy()
    
# cluster the topics
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(wdf_train_prime)
print("wdf_train, kmeans_inertia = ", kmeans.inertia_)

clusters = []
categories_prime = list(range(n_clusters))

y_train_prime = kmeans.predict(wdf_train_prime)
y_test_prime = kmeans.predict(wdf_test_prime)

for ii in range(n_clusters):
    clusters.append(Counter(y_train[y_train_prime == ii]))
    print(clusters[-1])

print()

wdf_train, kmeans_inertia =  4230.109369977496
Counter({'sci.space': 17, 'talk.politics.mideast': 16, 'rec.autos': 16, 'alt.atheism': 12})
Counter({'talk.politics.mideast': 3, 'alt.atheism': 2, 'rec.autos': 1})
Counter({'sci.space': 10, 'rec.autos': 8, 'talk.politics.mideast': 6, 'alt.atheism': 4})
Counter({'alt.atheism': 7, 'talk.politics.mideast': 4, 'sci.space': 4, 'rec.autos': 2})



In [27]:
wdf_train_prime.head()

,000,031349,10,1000,11,14,16th,1900,1940,1982,...,yeah,year,yes,yo,yorker,you,your,yourself,zeuge,__labels__
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [64]:
kmeans.predict(wdf_train_prime)

array([2, 0, 2, 2, 2, 0, 0, 3, 1, 2, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 3, 0,
       0, 0, 3, 3, 2, 0, 2, 2, 2, 3, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0,
       0, 1, 0, 3, 0, 3, 3, 2, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       3, 0, 0, 3, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 1, 2, 0, 0, 3, 3, 0, 0,
       0, 0, 3, 2, 3, 0, 0, 1, 0, 2, 0, 2, 1, 0, 1, 0, 0, 0, 0, 3, 0, 2,
       0, 3])

In [18]:
# the word topic distr
wdt_train = wdf_train_prime.copy()
wdt_test = wdf_test_prime.copy()

wdt_test["__labels__"] = y_test_prime
wdt_train["__labels__"] = y_train_prime

word_doc_count = wdf_train_prime.sum(0)
topic_doc_count = pd.DataFrame(Counter(y_train_prime), index=[0]).T[0]
word_topic_pr_distr = pd.DataFrame(data=0.0, columns=categories_prime, index=vocabulary)

for category in tqdm(categories_prime):
    pxy = wdf_train_prime[wdt_train["__labels__"] == category].sum(0) / topic_doc_count[category]
    word_topic_pr_distr[category] = pxy * (topic_doc_count[category] / word_doc_count)

print(f"word_topic_pr_distr shape = {word_topic_pr_distr.shape}")


word_topic_pr_distr shape = (1467, 4)


In [19]:
word_topic_pr_distr.head()

,0,1,2,3
000,1.0,0.0,0.0,0.0
031349,1.0,0.0,0.0,0.0
10,1.0,0.0,0.0,0.0
1000,1.0,0.0,0.0,0.0
11,0.0,0.0,1.0,0.0


In [62]:
word_topic_entropy = calculate_entropy(word_topic_pr_distr, axis=1)
word_topic_entropy_norm = (word_topic_entropy.max() - word_topic_entropy) / word_topic_entropy.max()

order_factor = word_topic_entropy_norm #* (word_doc_count / word_doc_count.max())

order_indices = order_factor.argsort()[::-1]
word_topic_pr_distr.iloc[order_indices].head(10)

,0,1,2,3
zeuge,1.0,0.0,0.0,0.0
ftp,0.0,0.0,1.0,0.0
full,0.0,1.0,0.0,0.0
fundraise,0.0,0.0,1.0,0.0
funny,1.0,0.0,0.0,0.0
further,1.0,0.0,0.0,0.0
fusi,1.0,0.0,0.0,0.0
gain,1.0,0.0,0.0,0.0
gallon,0.0,0.0,0.0,1.0
game,0.0,1.0,0.0,0.0


In [61]:
word = "space"
calculate_entropy(word_topic_pr_distr.loc[word]), npword_doc_count[word]

(0.6730116670092565, 5)

In [60]:
word_doc_count.mean

<bound method Series.mean of 000          2
031349       1
10           1
1000         1
11           2
            ..
yorker       1
you         87
your        23
yourself     1
zeuge        1
Length: 1467, dtype: int64>

In [20]:
# the entropy and softma for reducing relation
word_topic_pr_entropy = calculate_entropy(word_topic_pr_distr, axis=1)

# the norm freq of the word in topic
word_topic_pr_freq_weight = word_topic_pr_distr.sum(1)


# word_topic_pr_infl = 1 - softmax(word_topic_pr_entropy)

# influence
# wdf_train_prime *= word_topic_pr_infl

In [21]:
wdf_train_prime.max().max()

7

In [22]:
word_topic_pr_freq_weight.max()

1.0

In [23]:
word_topic_pr_entropy

array([0.       , 0.       , 0.       , ..., 1.0554635, 0.       ,
       0.       ])